In [1]:
import pandas as pd
import numpy as np
import os
import xarray as xr
import rioxarray
import re
import funciones_hidrologicas as fh

In [2]:
path = r'C:\Users\F-8143\OneDrive - SHI S.A.S\Libreria_hidrologia\datos_prueba\PTPM_CON@23055070.data'
df = fh.read_data(path)

In [3]:
def llenar_na(df:pd.DataFrame) -> pd.DataFrame:
    # Seleccion de fechas
    fecha_inicio = df.index[0]
    fecha_final = df.index[-1]
    # Generacion lista con fecha
    rango_fechas = pd.date_range(fecha_inicio,fecha_final,freq='d')
    # re-index de datos 
    df = df.reindex(rango_fechas)

    return df

def ciclo_anual(df:pd.DataFrame, umbral:float) -> pd.DataFrame:

    # Se rellena el dataframe con datos NA
    df_lleno = llenar_na(df)
    #Se agregan los datos faltantes de manera mensual
    df_faltantes = df_lleno.isna().resample('ME').sum()/31
    # Se agregan los datos de manera mensual
    df_mensual = df_lleno.resample('ME').sum()
    # Filtro de datos
    datos_mensual = df_mensual[df_faltantes < umbral]
    #Generacion del ciclo anual
    ciclo_anual = datos_mensual.groupby(datos_mensual.index.month).mean()
    # Estimacion de la media anual
    
    return ciclo_anual


In [4]:
ciclo_anual(df,0.1)

,23055070
1,349.748276
2,396.627586
3,510.853333
4,503.414286
5,472.327586
6,301.507692
7,291.137037
8,332.360000
9,437.548276
10,543.666667


In [ ]:
ruta = 

In [3]:
# Ejemplo de nombre de archivo
filename = "CHIRPS_1981-01-01.tif"

# Patrón de expresión regular
date_pattern = re.compile(r'CHIRPS_(\d{4}-\d{2}-\d{2})\.tif')

# Buscar el patrón en el nombre del archivo
match = date_pattern.search(filename)

# Extraer la fecha
if match:
    date_str = match.group(1)
    print(f"Fecha extraída: {date_str}")
else:
    print("No se encontró una fecha")


Fecha extraída: 1981-01-01


In [5]:
# Ruta de la carpeta que contiene los archivos GeoTIFF
input_folder = r'G:\Mi unidad\Chirps_Colombia_diario'

# Patrón de expresión regular para extraer la fecha del nombre del archivo
date_pattern = re.compile(r'CHIRPS_(\d{4}-\d{2}-\d{2})\.tif')

# Listar todos los archivos en la carpeta
tiff_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Crear una lista para almacenar los datasets y las fechas
datasets = []
dates = []

# Leer cada archivo y extraer la fecha
for tiff_file in tiff_files:
    match = date_pattern.search(tiff_file)
    if match:
        date_str = match.group(1)
        # Convertir la fecha a un objeto datetime
        date = pd.to_datetime(date_str, format='%Y-%m-%d')
        dates.append(date)
        # Leer el archivo GeoTIFF como un xarray.Dataset
        dataset = rioxarray.open_rasterio(os.path.join(input_folder, tiff_file))
        datasets.append(dataset)

# Concatenar los datasets a lo largo de la dimensión temporal
combined = xr.concat(datasets, dim='time')

# Asignar las fechas extraídas como coordenadas de la dimensión temporal
combined = combined.assign_coords(time=('time', dates))

# Guardar el dataset concatenado a un archivo NetCDF (o a otro formato si prefieres)
output_file = r'G:\Mi unidad'
combined.to_netcdf(output_file)

print(f'Archivo concatenado guardado en {output_file}')


KeyboardInterrupt: 